Load dataloader and embeddings.

In [ ]:
import torch

dataloader = torch.load('data/dataloader.pth')
embeddings = torch.load('data/embeddings.pt')

Apply t-SNE on the embeddings for 2D visualization.